In [17]:
import geopandas as gpd
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import KMeans
from sklearn.linear_model import LogisticRegressionCV
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import SMOTE
import shap
import warnings
warnings.filterwarnings('ignore')

In [18]:
# CONFIGURATION
BASE_PATH = "C:/Users/jason/OneDrive - The Pennsylvania State University/DAAN 881, Data Drive Decision Making/Project/Auto Data/Merged/"
SC_FILE = "sc_cleaned_combined_2013_2022.csv"
TN_FILE = "TN_PROCESSED_corrected_v2.csv"
PA_FILE = "PA_PROCESSED_corrected.csv"
FARS_FILE = "FARS_combined_cleaned_2013-2023_v2.csv"

SAVE_MERGED_FILE = True
MERGED_FILE_NAME = "merged_dataset.csv"

In [19]:
# DATA LOADING
def load_dataset(filepath):
    if os.path.exists(filepath):
        return pd.read_csv(filepath, dtype=str)
    else:
        print(f"File not found: {filepath}")
        return None

# Load datasets
sc_df = load_dataset(os.path.join(BASE_PATH, SC_FILE))
tn_df = load_dataset(os.path.join(BASE_PATH, TN_FILE))
pa_df = load_dataset(os.path.join(BASE_PATH, PA_FILE))
fars_df = load_dataset(os.path.join(BASE_PATH, FARS_FILE))

# Merge SC + TN + PA
merged_df = pd.concat([df for df in [sc_df, tn_df, pa_df] if df is not None], ignore_index=True)
if SAVE_MERGED_FILE:
    merged_df.to_csv(os.path.join(BASE_PATH, MERGED_FILE_NAME), index=False)
    print(f"Merged dataset saved as {MERGED_FILE_NAME}")

Merged dataset saved as merged_dataset.csv


In [20]:
# DATA PREP
def preprocess(df):
    df["crash_date"] = pd.to_datetime(df["crash_date"], errors="coerce")
    df["severity_level"] = pd.to_numeric(df["severity_level"], errors="coerce")
    df["driver_age"] = pd.to_numeric(df["driver_age"], errors="coerce")
    df["opioid_flag"] = pd.to_numeric(df["opioid_flag"], errors="coerce")
    df["alcohol_flag"] = pd.to_numeric(df["alcohol_flag"], errors="coerce")
    df["any_drug_flag"] = pd.to_numeric(df["any_drug_flag"], errors="coerce")
    return df

merged_df = preprocess(merged_df)
fars_df = preprocess(fars_df)

In [ ]:
# Imputation before modeling

#for feature in ['driver_age', 'alcohol_flag', 'any_drug_flag']:
#    if feature in train_df.columns:
#        missing_count = train_df[feature].isna().sum()
#        print(f"Missing {feature} in training set: {missing_count}")
        
#        if missing_count > 0:
#            if feature == 'driver_age':
#                median_age = train_df['driver_age'].median()
#                train_df['driver_age'] = train_df['driver_age'].fillna(median_age)
#                test_df['driver_age'] = test_df['driver_age'].fillna(median_age)
#            else:
#                train_df[feature] = train_df[feature].fillna(0)
#                test_df[feature] = test_df[feature].fillna(0)

In [21]:
# SAMPLE 5% TEST MODE (for safe testing first)
TEST_MODE = True
if TEST_MODE:
    merged_df = merged_df.sample(frac=0.05, random_state=42)
    fars_df = fars_df.sample(frac=0.05, random_state=42)
    print("Running in TEST MODE: Using only 5% of the data.")

Running in TEST MODE: Using only 5% of the data.


In [22]:
# SPLITTING
def split_data(df):
    df = df.sort_values('crash_date')
    train = df[(df['crash_date'].dt.year >= 2013) & (df['crash_date'].dt.year <= 2020)]
    valid = df[(df['crash_date'].dt.year == 2021)]
    test  = df[(df['crash_date'].dt.year == 2022)]
    return train, valid, test

train_df, valid_df, test_df = split_data(merged_df)

In [23]:
# BASELINE MODEL (Logistic Regression for Opioid Prediction)
def baseline_logistic(train_df, test_df):
    model = LogisticRegression(max_iter=1000)
    X_train = train_df[['driver_age', 'alcohol_flag', 'any_drug_flag']]
    y_train = train_df['opioid_flag']
    model.fit(X_train, y_train)

    X_test = test_df[['driver_age', 'alcohol_flag', 'any_drug_flag']]
    y_test = test_df['opioid_flag']
    preds = model.predict(X_test)

    print("\n=== Logistic Regression for Opioid Prediction ===")
    print(classification_report(y_test, preds))
    print("ROC-AUC:", roc_auc_score(y_test, model.predict_proba(X_test)[:,1]))

baseline_logistic(train_df, test_df)


=== Logistic Regression for Opioid Prediction ===
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     58568
           1       0.00      0.00      0.00        11

    accuracy                           1.00     58579
   macro avg       0.50      0.50      0.50     58579
weighted avg       1.00      1.00      1.00     58579

ROC-AUC: 0.7110289826278079


In [24]:
# RANDOM FOREST WITH SMOTE FOR SEVERITY
def random_forest_smote(train_df, test_df):
    smote = SMOTE()
    X_train = train_df[['driver_age', 'alcohol_flag', 'any_drug_flag']]
    y_train = train_df['severity_level']
    X_res, y_res = smote.fit_resample(X_train, y_train)

    rf = RandomForestClassifier(n_estimators=100, random_state=42)
    rf.fit(X_res, y_res)

    X_test = test_df[['driver_age', 'alcohol_flag', 'any_drug_flag']]
    y_test = test_df['severity_level']
    preds = rf.predict(X_test)

    print("\n=== Random Forest with SMOTE for Severity ===")
    print(classification_report(y_test, preds))

    return rf, X_test

rf_model, rf_X_test = random_forest_smote(train_df, test_df)

ValueError: Expected n_neighbors <= n_samples_fit, but n_neighbors = 6, n_samples_fit = 2, n_samples = 2

In [ ]:
# SHAP for Interpretability
def shap_explainer(model, X_test):
    explainer = shap.TreeExplainer(model)
    shap_values = explainer.shap_values(X_test)
    shap.summary_plot(shap_values, X_test)

shap_explainer(rf_model, rf_X_test)

In [ ]:
# PCA for Dimensionality Validation
def pca_analysis(df):
    features = ['driver_age', 'opioid_flag', 'alcohol_flag', 'any_drug_flag', 'fatalities', 'injuries']
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(df[features].fillna(0))

    pca = PCA()
    pca.fit(X_scaled)

    plt.plot(np.cumsum(pca.explained_variance_ratio_), marker='o')
    plt.title('PCA Scree Plot')
    plt.xlabel('Number of Components')
    plt.ylabel('Cumulative Explained Variance')
    plt.grid()
    plt.show()

pca_analysis(merged_df)

In [ ]:
# PRESCRIPTIVE RECOMMENDATION ENGINE
def prescriptive_analysis(df):
    df['high_risk_driver'] = ((df['opioid_flag'] == 1) | (df['alcohol_flag'] == 1) | (df['driver_age'] < 21) | (df['driver_age'] > 70)).astype(int)

    risk_summary = df.groupby('county_fips')['high_risk_driver'].mean().sort_values(ascending=False)

    plt.figure(figsize=(12,6))
    risk_summary.head(20).plot(kind='barh')
    plt.title('Top 20 Counties by High Risk Driver Rates')
    plt.xlabel('Proportion of High-Risk Drivers')
    plt.gca().invert_yaxis()
    plt.grid()
    plt.show()

prescriptive_analysis(merged_df)

In [ ]:
# FARS side-by-side
print("\n\n=============\nRunning Same Pipelines on FARS\n=============")
train_fars, valid_fars, test_fars = split_data(fars_df)
baseline_logistic(train_fars, test_fars)
rf_model_fars, rf_X_test_fars = random_forest_smote(train_fars, test_fars)
shap_explainer(rf_model_fars, rf_X_test_fars)
pca_analysis(fars_df)
prescriptive_analysis(fars_df)

print("\n\nALL FINAL SCRIPTS COMPLETE")